In [2]:
%load_ext autoreload
%autoreload 2

In [173]:
import sys
import subprocess
import os
import pandas as np
import numpy as np
import pathlib
import re
import glob
import pandas as pd
from pprint import pprint
module_path = "/Users/jmccarty/Data/221205_ipv_workbench/github/IPV_Workbench"
sys.path.insert(0, module_path)
from workbench.utilities import general, config_utils, temporal, io
from workbench.manage import manage, host
# from workbench.manage import host
from workbench.workflows import workflows

# Overview

# Part 1 Initialise and Build Model

## 1.1 Initialise

In [3]:
# initialise the project folder
# this creates a project folder in the parent dir using the project name and the base EPW file
# the most important result of this process is the config file

parent_dir = os.path.join("/Users","jmccarty","Desktop","idp_tutorial")
project_name = "idp_intro_day"
project_epw = os.path.join("/Users","jmccarty","Downloads", "fluntern_2001-2017.epw")

config_path = manage.initiate_project(parent_dir, project_name, project_epw)


<div style="align-contents: center; width: 600px;">
<img src="../../../sketches/folder_struct_new config.png" alt="Basic Config File" style="float:left; width: 100%;"/>
<p style="text-align: center; margin: 0 auto; width: 90%;"> The initial folder structure
</p>
</div>


<div style="align-contents: center; width: 600px;">
<img src="../../../sketches/basic_config_file.png" alt="Basic Config File" style="float:left; width: 100%;"/>
<p style="text-align: center; margin: 0 auto; width: 90%;"> Here we can see the default config file that was just created. There are several sections and under each paremters that we will adjust throughout the tutorial using an instance of a Python object that is meant for managing the project.
</p>
</div>

In [4]:
# intialise the project manager object using the config file
# the basic operations in the project manager is the creation of file paths associated to the object which makes writing files, getting input data, 
# and accessing project details much easier
project_manager = manage.Project(config_path)

# for example when the projec tis initiated it reads the location details of the EPW file and enters them into the config 
print("Project latitude: ", project_manager.management_latitude)
print("Project longitude: ", project_manager.management_longitude)

Project latitude:  47.378
Project longitude:  8.566


In [5]:
# Feature Example
# you can directly manipulate the config file using the edit_cfg_file class method of the project manager
# you just need to enter the secion (in brackets in the file), the key (what you want to edit), and the new value

# in this example we will change the host name (we will cover hosts later, but this is essentially a building name)
print(project_manager.management_host_name)
project_manager.edit_cfg_file("management", "host_name", "B1008")
print(project_manager.management_host_name)

B1000
B1008


In [6]:
# the project manager contains a key method for setting up the rest of the project strucutre and th einitial files
# there are several base files stored in your copy of CACTUS, they will be copied into a new file driectory
project_manager.project_setup()

The project is initialized. We have created a base host object named 'B1008'.
You will need to either move or create the geometry and panelizer files into the appropriate directories.
The geometry files should follow the convention defined in the output of the template grasshopper and rhino files.
The panelizer files are those that have been created using the grasshopper utility. The Panelizer is
not ready for a pure python implementation as of yet.



<div style="align-contents: center; width: 600px;">
<img src="../../../sketches/folder_struct_initialised project.png" alt="Basic Config File" style="float:left; width: 100%;"/>
<p style="text-align: center; margin: 0 auto; width: 90%;"> Upon intialising the project a large file structure will be created. Importantly some template files will be added to ghe geometry subfolder that we can use to build our own project.
</p>
</div>

## 1.2 Model Builder

<div style="align-contents: center; width: 600px;">
<img src="../../../sketches/cactus_template_geo.png" alt="Basic Config File" style="float:left; width: 100%;"/>
<p style="text-align: center; margin: 0 auto; width: 90%;"> In its current form CACTUS still requires the user to rely on Rhino and Grasshopper for building the 3D model. This will change in future versions. The provided templates simplify the process by showing a small example that can be replaced. Here we see the template file "0_cactus_geometry_template.3dm" and the corresponding Grasshopper file.
</p>
<br>
<p style="text-align: left; margin: 0 auto; width: 90%;"> I have tried to reduce the number of dependencies for the Grasshopper scripts but several additional packages are required: <br \>
 - Pancake: https://www.food4rhino.com/en/app/pancake  <br \>
 - Human: https://www.food4rhino.com/en/app/human
</p>
</div>

<div style="align-contents: center; width: 400px;">
<img src="../../../sketches/duplicate_templates.png" alt="Basic Config File" style="float:left; width: 100%;"/>
<p style="text-align: center; margin: 0 auto; width: 90%;"> Because these are template files and can potentially be overwritten if the project_manager.project_setup method is called again the first thing to do is make copies and rename or saveas. Then open the renamed files that gein with "0_". This is the geometry modeler.
</p>
</div>

<div style="align-contents: center; width: 400px;">
<img src="../../../sketches/set project config.png" alt="Basic Config File" style="float:left; width: 100%;"/>
<p style="text-align: center; margin: 0 auto; width: 90%;"> The first step is to link the config file to the geometry modeling workflow by setting the Project Config File (File Path) component to the location of your config file. 
</p>
</div>

<div style="align-contents: center; width: 400px;">
<img src="../../../sketches/geometry_builder_export.png" alt="Basic Config File" style="float:left; width: 100%;"/>
<p style="text-align: center; margin: 0 auto; width: 90%;"> Each group on the left side of the grasshopper definition provides a specific function for intaking geometry, manipulating it based on its type, and feeding it to the export component, which is shown here. This is the objective of the gemoetry builder template. 
</p>
</div>

<div style="align-contents: center; width: 400px;">
<img src="../../../sketches/geometry_builder_host.png" alt="Basic Config File" style="float:left; width: 100%;"/>
<p style="text-align: center; margin: 0 auto; width: 90%;"> The object that contains the surfaces you want to apply PV panels to is known as the host. In this case that is the building highted in green. The grasshopper file uses a pipeline component to intake objects that are drawn in Rhino and stored in specific layers. 
</p>
</div>

<div style="align-contents: center; width: 400px;">
<img src="../../../sketches/geometry_builder_rename_host.png" alt="Basic Config File" style="float:left; width: 100%;"/>
<p style="text-align: center; margin: 0 auto; width: 90%;"> Note that we changed the host name to B1008 in the config file in an earlier cell. We need to make this adjustment in the grasshopper tempalte as well.
</p>
</div>

<div style="align-contents: center; width: 400px;">
<img src="../../../sketches/geometry_builder_layers.png" alt="Basic Config File" style="float:left; width: 100%;"/>
<p style="text-align: center; margin: 0 auto; width: 90%;"> Here we show the basic layers that the template comes with and is prepared to intake. Extending past these is not recommended at this time.
</p>
</div>

<div style="align-contents: center;  width: 800px; height: 250px; align-contents: middle">
<img src="../../../sketches/geometry_builder_terrain.png" alt="Basic Config File" style="float:left; width: 33%;"/>
<img src="../../../sketches/geometry_builder_host_objects.png" alt="Basic Config File" style="float:left; width: 33%;"/>
<img src="../../../sketches/geometry_builder_opaque_context.png" alt="Basic Config File" style="float:left; width: 33%;"/>
</div>
<div style="align-contents: center; width: 800px; height: 150px; align-contents: middle">
<img src="../../../sketches/geometry_builder_seasonal_vegetation.png" alt="Basic Config File" style="float:left; width: 50%;"/>
<img src="../../../sketches/geometry_builder_constant_vegetation.png" alt="Basic Config File" style="float:left; width: 50%;"/>
</div>
<div style="align-contents: center; width: 800px">
<p style="text-align: center; margin: 0 auto; width: 90%;">Other than the host here are the other layers. Left to right: terrain, context objects on the host, surrounding opaque context (shading), vegetation that sheds leaves throughout the year, vegetation that does not shed leaves.
</p>
</div>


<div style="align-contents: center; width: 800px;">
<img src="../../../sketches/geometry_builder_annotations.png" alt="Basic Config File" style="float:left; width: 100%;"/>
<p style="text-align: center; margin: 0 auto; width: 90%;"> The template does not necessarily know which surfaces on the host that it is supposed to keep for the future panelizaer and analysis. Therefore there is a component group that allows the user to set a list corresponding to the surfaces. For each surface find the text tag and enter into the list the correct category under the same index order.
</p>
</div>

<div style="align-contents: center; width: 800px;">
<img src="../../../sketches/geometry_builder_surface_builder.png" alt="Basic Config File" style="float:left; width: 100%;"/>
<p style="text-align: center; margin: 0 auto; width: 90%;"> The last piece of the builder deals with surfaces that may have non integrated arrays. In this case that is the rooftop array. The sliders control the parameters of the array such that the user is free to manipulate it however they would like.
</p>
</div>

<div style="align-contents: center; width: 800px;">
<img src="../../../sketches/geometry_builder_new_surfaces.png" alt="Basic Config File" style="float:left; width: 100%;"/>
<p style="text-align: center; margin: 0 auto; width: 90%;"> It is understandable that outside of these layers you may want to add some geometry to act as a surface for PV. Luckily, with the flexibility of Grasshopper this is possible. Here we add several surfaces in the form of fixed shading devices that a PV module could be placed on. Note the new layer "new_surfaces".
</p>
</div>
<br>
<div style="align-contents: center; width: 800px;">
<img src="../../../sketches/geometry_builder_add_surface_A.png" alt="Basic Config File" style="float:left; width: 50%;"/>
<img src="../../../sketches/geometry_builder_add_surface_B.png" alt="Basic Config File" style="float:left; width: 50%;"/> 
<br>
<p style="text-align: center; margin: 0 auto; width: 90%;"> Once the new surfaces are pulled into Grasshopper using the newly added pipeline component we can conncet them into the flow. At the far right of the script is a Merge component to add the surfaces to.
</p>
</div>

<div style="align-contents: center; width: 800px;">
<img src="../../../sketches/geometry_builder_export_False.png" alt="Basic Config File" style="float:left; width: 50%;"/>
<img src="../../../sketches/geometry_builder_export_True.png" alt="Basic Config File" style="float:left; width: 50%;"/> 
<br>
<p style="text-align: center; margin: 0 auto; width: 90%;"> Once finished with the geometry builder the model can be exported to a fresh rhino file using the boolean attached to the export component.
</p>


<div style="align-contents: center; width: 600px;">
<img src="../../../sketches/folder_struct_geometry_done.png" alt="Basic Config File" style="float:left; width: 100%;"/>
<p style="text-align: center; margin: 0 auto; width: 90%;"> The newly created geometry file.
</p>
</div>

## 1.3 Panelizer


<div style="align-contents: center; width: 800px;">
<img src="../../../sketches/panelizer_overview_coarse.png" alt="Basic Config File" style="float:left; width: 100%;"/>
<p style="text-align: center; margin: 0 auto; width: 90%;"> The Panelizer is a mostly automated Grasshopper script which intakes details about a potential PV module and automatically generates an array on the specified surfaces. The key output is a file type known as a pickle which is a Python native format. This pickle describes the details of the various arrays for a Host object. The file is used to relate irradiance data from the irradiance workflow, which the Panelizer script is also responble for generating the properly formattted Radiance folder that is used for the raytracing. This Grasshopper version is a Beta version of the Panelizer, which will move to pure python in the coming months. 
</p>
</div>

<div style="align-contents: center; width: 800px;">
<img src="../../../sketches/typical_devices.png" alt="Basic Config File" style="float:left; width: 50%;"/>
<img src="../../../sketches/bipv_co2_impact.png" alt="Basic Config File" style="float:left; width: 50%;"/> 

<br>
<p style="text-align: center; margin: 0 auto; width: 90%;"> The first thing to do select a device to work from. Included with CACTUS are several "typical" devices that were charatersied from a large database of modules on the market from the past 5 years. It is possible to create new devices, but we will go over that in a later tutorial. Here you can see the typical devices on the left and then how they relate to emboided carbon from Galimshina et al. (2023).
</p>
</div>

In [179]:
# because we have restarted our notebook we will reactivate our project by reading in the config path
# place your config path here
config_path = "/Users/jmccarty/Data/221205_ipv_workbench/tutorials/cactus_tutorial/cactus_tutorial.config"
project_manager = manage.Project(config_path)

# rerun the setup in order to rebuild the entire project object and its attributes
project_manager.project_setup()


Destination file already exists, copy aborted./n/Users/jmccarty/Data/221205_ipv_workbench/tutorials/cactus_tutorial/inputs/shared/tmy/base.epw
Destination file already exists, copy aborted./n/Users/jmccarty/Data/221205_ipv_workbench/tutorials/cactus_tutorial/inputs/hosts/B1008/base/geometry/0_cactus_geometry_template.3dm
Destination file already exists, copy aborted./n/Users/jmccarty/Data/221205_ipv_workbench/tutorials/cactus_tutorial/inputs/hosts/B1008/base/geometry/0_cactus_geometry_template.gh
Destination file already exists, copy aborted./n/Users/jmccarty/Data/221205_ipv_workbench/tutorials/cactus_tutorial/inputs/hosts/B1008/base/geometry/1_cactus_panelizer_template.gh
The project is initialized. We have created a base host object named 'B1008'.
You will need to either move or create the geometry and panelizer files into the appropriate directories.
The geometry files should follow the convention defined in the output of the template grasshopper and rhino files.
The panelizer files

In [ ]:
# wee need to input the device id into our config file
# in the shared folder of our project you will find CSV file for typical devices "cactus_typical_devices" 
# let's see what we have "cactus_tutorial/inputs/shared/cell_module_data/cactus_typical_devices.csv"

In [180]:
# we need to change the device id config option so grasshopper/panelizer can read it
print(project_manager.analysis_device_id)
project_manager.edit_cfg_file("analysis", "device_id", "D003")
print(project_manager.analysis_device_id)

D003
D003



<div style="align-contents: center; width: 500px;">
<img src="../../../sketches/panelizer_tutorial/panelizer_files.png" alt="Basic Config File" style="float:left; width: 100%;"/>
<p style="text-align: center; margin: 0 auto; width: 90%;"> Now we can open the Panelizer files. Start with the rhino file and then load the grasshopper.
</p>
</div>


<div style="align-contents: center; width: 800px;">
<img src="../../../sketches/panelizer_tutorial/panelizer_overview.png" alt="Basic Config File" style="float:left; width: 100%;"/>
<p style="text-align: center; margin: 0 auto; width: 90%;"> The panelizer has several parts. The first two are separated vertically. The upper workflow is for the actual panelization process. The smaller lower section is purely for writing a Radiance simulation project that CACTUS can read.
</p>
</div>

In [8]:
# show slideshow

In [181]:
building = host.Host(project_manager)

In [182]:
workflows.run_irradiance(building)

Starting Radiance workflow for surface 1008_4.
 - Running 2-Phase DDS with 7 workers
 - Current surface is 1008_4
 - Initializing the opening weather file.
 - Starting Part 1 (total).
     - oconv
     - rfluxmtx
     - gendaymtx
     - dctimestep | rmtxop
 - Starting Part 2 (direct).
     - oconv
     - rfluxmtx
     - gendaymtx
     - dctimestep | rmtxop
 - Starting Part 3 (sun).
     - create_primitive_sun
     - rcalc
     - oconv
     - rcontrib
     - gendaymtx
     - dctimestep | rmtxop
Completed in 62.54 seconds.
Converting .ill files to feather for surface 1008_4.
 - Saving Irradiance results
    - Direct sensor data saved in compressed format, time=0.0-seconds.
    - Diffuse sensor data saved in compressed format, time=0.0-seconds.
Completed in 0.47 seconds.
-----------------------
Starting Radiance workflow for surface 1008_3.
 - Running 2-Phase DDS with 7 workers
 - Current surface is 1008_3
 - Initializing the opening weather file.
 - Starting Part 1 (total).
     - oconv


In [183]:
workflows.run_module_point(building, "center_point")

Starting module center_point analysis workflow for surface 1008_4.
{1008;4}
Completed in 0.12 seconds.
-----------------------
Starting module center_point analysis workflow for surface 1008_3.
{1008;3}
Completed in 0.12 seconds.
-----------------------
Starting module center_point analysis workflow for surface 1008_7.
{1008;7}
Completed in 0.27 seconds.
-----------------------
Starting module center_point analysis workflow for surface 1008_8.
{1008;8}
Completed in 0.16 seconds.
-----------------------
Starting module center_point analysis workflow for surface 1008_10.
{1008;10}
Completed in 0.12 seconds.
-----------------------
Starting module center_point analysis workflow for surface 1008_14.
{1008;14}
Completed in 1.63 seconds.
-----------------------
Starting module center_point analysis workflow for surface 1008_12.
{1008;12}
Completed in 2.21 seconds.
-----------------------
Starting module center_point analysis workflow for surface 1008_0.
{1008;0}
Completed in 0.07 seconds.
--

In [184]:
workflows.run_module_point(building, "cell_point")

Starting module cell_point analysis workflow for surface 1008_4.
Completed in 0.25 seconds.
-----------------------
Starting module cell_point analysis workflow for surface 1008_3.
Completed in 0.35 seconds.
-----------------------
Starting module cell_point analysis workflow for surface 1008_7.
Completed in 1.16 seconds.
-----------------------
Starting module cell_point analysis workflow for surface 1008_8.
Completed in 0.68 seconds.
-----------------------
Starting module cell_point analysis workflow for surface 1008_10.
Completed in 0.36 seconds.
-----------------------
Starting module cell_point analysis workflow for surface 1008_14.
Completed in 7.33 seconds.
-----------------------
Starting module cell_point analysis workflow for surface 1008_12.
Completed in 9.2 seconds.
-----------------------
Starting module cell_point analysis workflow for surface 1008_0.
Completed in 0.18 seconds.
-----------------------
Starting module cell_point analysis workflow for surface 1008_9.
Compl

In [174]:
pprint(building.get_dict_instance(["{1008;10}","{1008;10;7}"]))

{'CellsIrradDiffuse': None,
 'CellsIrradDirect': None,
 'CellsIrradEff': None,
 'CellsNormals': [[0.283, -0.1228, 0.9512],
                  [0.283, -0.1228, 0.9512],
                  [0.283, -0.1228, 0.9512],
                  [0.283, -0.1228, 0.9512],
                  [0.283, -0.1228, 0.9512],
                  [0.283, -0.1228, 0.9512]],
 'CellsTemp': None,
 'CellsXYZ': [[11.133419, 12.957406, 6.420086],
              [11.130367, 12.808559, 6.401777],
              [10.989585, 12.955126, 6.462591],
              [10.986533, 12.806279, 6.444281],
              [10.845751, 12.952847, 6.505095],
              [10.842699, 12.804, 6.486786]],
 'Curves': {'Imod': {}, 'Vmod': {}},
 'Details': {'panelizer_cec_key': None,
             'panelizer_cell_type': 'monoSi',
             'panelizer_center_pt': [10.974315, 12.853205, 6.453975],
             'panelizer_module_type': 'D009',
             'panelizer_n_cells': 6,
             'panelizer_n_cols': 2,
             'panelizer_n_cols_ideal':